# Trabalho Prático 3
## ISEL - LEIM

## Docente - Gonçalo Marques

### Trabalho realizado por:
Mª Luísa Melo e Sampaio Nº50799
André Nº
Carlos Ceita Nº50950

In [1]:
#imports
from time import time
from os import path
import numpy as np
import cv2
import matplotlib.pyplot as plt
import heapq
from collections import defaultdict

### Exercicio 1

In [ ]:
def gen_huff_table(simbolos):
    """Função que cria a tabela de códigos de Huffman a partir de um dicionário 

    Args:
        simbolos (dicionário): dicionário com as frequências de cada letra

    Returns:
        dicionário
    """
    # Criar heap de nós: [peso, [[símbolo, código]]]
    heap = [[peso, [[simbolo, ""]]] for simbolo, peso in simbolos.items()]
    heapq.heapify(heap)

    # Construir árvore de Huffman
    while len(heap) > 1:
        peso_lo, lista_lo = heapq.heappop(heap)
        peso_hi, lista_hi = heapq.heappop(heap)

        # Prefixos: 0 para o ramo de menor peso, 1 para o ramo de maior peso
        for par in lista_lo:
            par[1] = '0' + par[1]
        for par in lista_hi:
            par[1] = '1' + par[1]

        # Reunir em novo nó
        novo_no = [peso_lo + peso_hi, lista_lo + lista_hi]
        heapq.heappush(heap, novo_no)

    # Extrair tabela final
    _, tabela = heapq.heappop(heap)
    # Ordenar por comprimento e lexicograficamente
    tabela.sort(key=lambda par: (len(par[1]), par[0]))

    # Converter para dicionário {símbolo: código}
    return {simbolo: codigo for simbolo, codigo in tabela}

In [16]:
dic = {'o': 5, 'r': 3, 'i': 3, 't': 2, 'n': 2, 'l': 2, 'a': 2, 'g': 2, 's': 1} # otorrinolaringologista
# não dá exatamente a mesma do que nos slides, perguntar ao stor se é normal

tabela = gen_huff_table(dic)

print(tabela) 

mensagem = ""
for letra, indice in dic.items():
    mensagem += letra * indice

print("mensagem: ", mensagem)


{'o': '01', 'i': '100', 'n': '000', 'r': '101', 't': '001', 'a': '1101', 'g': '1110', 'l': '1111', 's': '1100'}
mensagem:  ooooorrriiittnnllaaggs


### Exercício 2

In [ ]:
def encode_huff(mensagem, tabela):
    """Dada uma mensagem e a tabela do ponto anterior, retorna uma sequência de bits com mensagem codificada.
    
    Args:
        mensagem (String): mensagem a codificar
        tabela (dicionário): tabela gerada com gen_huff_table
        
    Returns:
        Mensagem codificada em bits
    """
    mensagem_codificada = ''.join(tabela[simbolo] for simbolo in mensagem)
    bits_total = len(mensagem_codificada)
    return mensagem_codificada, bits_total
    

In [18]:
#exemplo de uso
mensagem_codificada, tamanho_total = encode_huff(mensagem=mensagem, tabela=tabela)

print("Tabela de Huffman")
for simbolo, codigo in tabela.items():
    print(f"{simbolo}: {codigo}")
    
print(f"\nMensagem codificada: \n{mensagem_codificada}")
print(f"\nTamanho total da mensagem codificada em bits: {tamanho_total}")

Tabela de Huffman
o: 01
i: 100
n: 000
r: 101
t: 001
a: 1101
g: 1110
l: 1111
s: 1100

Mensagem codificada: 
01010101011011011011001001000010010000001111111111011101111011101100

Tamanho total da mensagem codificada em bits: 68


### Exercício 3

In [ ]:
def decode_huff(mensagem_codificada, tabela):
    """
    Dada uma sequência de bits e a tabela gerada com a primeira função, retornar uma sequência de símbolos.
    Garante que a mensagem retornada é igual à mensagem dada como parâmetro de entrada da função anterior.
    
    Args:
        mensagem_codificada (String): mensagem codificada acima
        tabela (dict): Dicionário de huffman
        
    Returns:
        Mensagem sequência de símbolos
    """
    # inverter tabela para mapear codigos a simbolos
    tabela_invertida = {codigo: sim for sim, codigo in tabela.items()}
    descodificada = ""
    buffer = ""
    for bit in mensagem_codificada:
        buffer += bit
        if buffer in tabela_invertida:
            descodificada += tabela_invertida[buffer]
            buffer = ""
    return descodificada
    

In [22]:
# exemplos de uso
mensagem_descodificada = decode_huff(mensagem_codificada=mensagem_codificada, tabela=tabela)
print(mensagem_descodificada)

ooooorrriiittnnllaaggs


### Exercício 4

In [23]:
def encode_huff_table(tabela):
    """
    Dada a tabela huffman do ponto 1, retorna uma sequência de bits correspondente à tabela codificada
    Args:
        tabela: Tabela do ponto 1
    
    Returns:
        String binária com a tabela codificada
    """
    tabela_codificada = ""
    for simbolo, codigo in tabela.items():
        simbolo_bin = format(ord(simbolo), '08b') #simbolo em 8 bits ascii
        tamanho_codigo_bin = format(len(codigo), '05b') # comprimento do código em 5 bits
        tabela_codificada += simbolo_bin + tamanho_codigo_bin + codigo # juntar tudo
    return tabela_codificada

In [26]:
# exemplos de uso
sequencia_tabela = encode_huff_table(tabela=tabela)
sequencia_final = sequencia_tabela + mensagem_codificada
print("Sequência codificada da tabela:")
print(sequencia_tabela)

print("\nSequência final (tabela + mensagem):")
print(sequencia_final)

print(f"\nTamanho total: {len(sequencia_final)} bits")


Sequência codificada da tabela:
011011110001001011010010001110001101110000110000111001000011101011101000001100101100001001001101011001110010011100110110000100111101110011001001100

Sequência final (tabela + mensagem):
01101111000100101101001000111000110111000011000011100100001110101110100000110010110000100100110101100111001001110011011000010011110111001100100110001010101011011011011001001000010010000001111111111011101111011101100

Tamanho total: 215 bits


In [ ]:
# ler um dos ficheiros
x = np.fromfile("LenaGray.tif", dtype=uint8)

# calcular histograma
h, bins, patches = plt.hist(x, 256, [0, 256])

# gerar código huffman
to = time()
tabela_codigo = gen_huff_table(np.arange(0,256), h)

t1 = time()
print("Time: ", t1 - to)

# codificar a mensagem
seq_bit0 = encode_huff(x, tabela_codigo)

# codificar tabela
seq_bit1 = encode_huff_table(tabela_codigo)

# concatenar sequencia de bits fa tabela e da mensagem
...

# escrver ficheiro
write2file(..., filename)
t2 = time()
print("Time: ", t2 - t1)

# ler ficheiro e descodificar a tabela
tabela_codigo, seq_bit1 = read_file(filename)

# descodificar mensagem
yi = decode_huff(seq_bit1, tabela_codigo)
t3 = time()
print("Time: ", t3 - t2)

size_ini = path.getsize("filename_original_image")
size_end = path.getsize("filename_compressed")
print("Taxa: ", 1. * size_ini/size_end)